In [2]:
import warnings
import alphalens
import pandas as pd
from scipy import stats
from scipy.stats import ConstantInputWarning
import sys
sys.path.insert(0, "../")
from alphalens import performance as perf
import dolphindb as ddb 

from factor_cal.config_loader import basic_config as cfg
from factor_cal.table.ddb_table import PriceTable, SecLevelFacTable
from factor_cal.utils import ddb_utils as du


# 忽略特定类别的警告
warnings.filterwarnings("ignore", category=ConstantInputWarning)

# read config file
config = cfg.BasicConfig('config/config.yml')
factor_name = "pv_corr"
# obtain the ddb session
s = du.DDBSessionSingleton().session


In [2]:
def get_clean_data(factor_name, date):
    fac_tb = SecLevelFacTable(config['factor_dbPath'], config['factor_tbName'])
    fac = fac_tb.load_factor(factor_name, date, config['start_time'], config['end_time'], sec_list=None)
    
    price_info = config['price_info']
    pc_tb = PriceTable(price_info['price_dbPath'], price_info['price_tbName'], price_info['time_col'], price_info['sec_col'], price_info['price_cols'])
    price = pc_tb.load_price(date, config['start_time'], config['end_time'], sec_list=None)
    
    fac_df = s.loadTable(tableName=fac).toDF()
    if (fac_df.empty):
        return None
    price_df = s.loadTable(tableName=price).toDF()
    
    fac_df = fac_df.set_index(['tradetime', 'securityid'])
    fac_df = fac_df['value']
    fac_df = fac_df.sort_index(level=0)
    
    price_df = price_df.set_index(['tradetime', 'securityid'])
    prices = price_df['close'].unstack()
    
    data=alphalens.utils.get_clean_factor_and_forward_returns(
        fac_df, prices, quantiles=5,periods=(20,60,100))
    return data

In [3]:
data3 = get_clean_data(factor_name, '2023.09.22')


/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/alphalens/utils.py:158: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series(index=x.index)
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f6e34e8fcd0>>
Traceback (most recent call last):
  File "/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
  File "_pydevd_bundle/pydevd_cython.pyx", line 1457, in _pydevd_bundle.pydevd_cython.SafeCallWrapper.__call__
  File "_pydevd_bundle/pydevd_cython.pyx", line 1758, in _pydevd_bundle.pydevd_cython.ThreadTracer.__call__
  File "/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydev_bundle/pydev_is_thread_alive.py", line 9

Dropped 98.9% entries from factor data: 30.6% in forward returns computation and 68.3% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).


MaxLossExceededError: max_loss (35.0%) exceeded 98.9%, consider increasing it.

In [ ]:
data3

In [6]:
date1 = '2023.09.22'
date2 = '2024.02.07'
data1 = get_clean_data(factor_name, date1)


/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/alphalens/utils.py:158: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series(index=x.index)


Dropped 9.2% entries from factor data: 8.4% in forward returns computation and 0.9% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!


In [7]:
data1

1m        3m        5m    factor  \
date                asset                                               
2023-09-22 09:45:00 000002.SZ -0.000755  0.000000  0.000000 -0.003759   
                    000004.SZ  0.005611  0.000000 -0.002494 -0.018960   
                    000006.SZ -0.002174 -0.002174  0.002174  0.004367   
                    000010.SZ -0.006006 -0.009009 -0.012012  0.000000   
                    000026.SZ  0.000000  0.002636  0.003515 -0.006981   
...                                 ...       ...       ...       ...   
2023-09-22 14:55:00 688799.SH  0.001240  0.001240  0.001240 -0.006652   
                    688800.SH  0.000000 -0.000509 -0.000509 -0.001272   
                    688819.SH  0.000286  0.000859  0.000859  0.003448   
                    688981.SH  0.000000  0.000586  0.000586 -0.000975   
                    689009.SH  0.000856  0.000571  0.000571  0.000571   

                               factor_quantile  
date                asset                       
2023-09-22 09:45:00 000002.SZ              2.0  
                    000004.SZ              1.0  
                    000006.SZ              3.0  
                    000010.SZ              3.0  
                    000026.SZ              2.0  
...                                        ...  
2023-09-22 14:55:00 688799.SH              1.0  
                    688800.SH              1.0  
                    688819.SH              5.0  
                    688981.SH              1.0  
                    689009.SH              3.0  

[22051635 rows x 5 columns]

In [8]:
data2 = get_clean_data(factor_name, date2)

/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/alphalens/utils.py:158: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series(index=x.index)


Dropped 8.4% entries from factor data: 8.4% in forward returns computation and 0.1% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!


In [10]:
data = pd.concat([data1, data2], axis=0)

In [13]:

group_neutral = False
ic_data = perf.factor_information_coefficient(data, group_neutral)


ic_summary_table = pd.DataFrame()
ic_summary_table["IC Mean"] = ic_data.mean()
ic_summary_table["IC Std."] = ic_data.std()
ic_summary_table["Risk-Adjusted IC"] = \
    ic_data.mean() / ic_data.std()
t_stat, p_value = stats.ttest_1samp(ic_data, 0, nan_policy='omit')
ic_summary_table["t-stat(IC)"] = t_stat
ic_summary_table["p-value(IC)"] = p_value
ic_summary_table["IC Skew"] = stats.skew(ic_data, nan_policy='omit')
ic_summary_table["IC Kurtosis"] = stats.kurtosis(ic_data, nan_policy='omit')
ic_summary_table['IC win rate'] = (ic_data > 0).sum() / ic_data.count()

In [14]:
ic_summary_table

,IC Mean,IC Std.,Risk-Adjusted IC,t-stat(IC),p-value(IC),IC Skew,IC Kurtosis,IC win rate
1m,-0.094709,0.099195,-0.954777,-89.357090,0.0,0.434262,0.388191,0.168512
3m,-0.085754,0.127504,-0.672560,-62.944589,0.0,0.284041,0.030687,0.243407
5m,-0.071779,0.137860,-0.520664,-48.728696,0.0,0.167942,-0.102550,0.301861


In [15]:
save_path = '/home/wangzirui/workspace/factor_ic_summary'
ic_summary_table.to_pickle(f'{save_path}/{factor_name}.pkl')

In [25]:
result = pd.read_pickle(f'{save_path}/{factor_name}.pkl')

In [26]:
result

,IC Mean,IC Std.,Risk-Adjusted IC,t-stat(IC),p-value(IC),IC Skew,IC Kurtosis,IC win rate
1m,-0.110157,0.095285,-1.156078,-254.026932,0.0,0.799751,2.929081,0.108322
3m,-0.116683,0.134783,-0.865706,-190.222987,0.0,0.694428,2.763240,0.153867
5m,-0.111498,0.148553,-0.750564,-164.922682,0.0,0.741072,2.938257,0.180668


In [2]:

fac_tb = SecLevelFacTable(config['factor_dbPath'], config['factor_tbName'])
fac = fac_tb.load_factor(factor_name, "2023.09.22", config['start_time'], config['end_time'], sec_list=None)

In [3]:
price_info = config['price_info']
pc_tb = PriceTable(price_info['price_dbPath'], price_info['price_tbName'], price_info['time_col'], price_info['sec_col'], price_info['price_cols'])
price = pc_tb.load_price("2023.09.22", config['start_time'], config['end_time'], sec_list=None)

In [4]:
price_df = s.loadTable(tableName=price).toDF()
fac_df = s.loadTable(tableName=fac).toDF()

In [5]:
fac_df = fac_df.set_index(['tradetime', 'securityid'])
fac_df = fac_df['value']
fac_df = fac_df.sort_index(level=0)

In [6]:
price_df = price_df.set_index(['tradetime', 'securityid'])
prices = price_df['close'].unstack()

In [12]:
start_time = pd.to_datetime('9:45').time()
end_time = pd.to_datetime('14:30').time()

prices = prices.between_time(start_time, end_time)

In [26]:
start_time = pd.to_datetime('9:45').time()
end_time = pd.to_datetime('13:30').time()

test_df = fac_df.reset_index().set_index('tradetime').between_time(start_time, end_time)
test_df.reset_index(inplace=True)
test_df.set_index(['tradetime','securityid'], inplace=True)
# selected_data = fac_df.between_time(start_time, end_time)

value
tradetime           securityid          
2023-09-22 09:45:00 000001.SZ        NaN
                    000002.SZ   0.278530
                    000004.SZ   0.284619
                    000005.SZ  -0.152224
                    000006.SZ        NaN
...                                  ...
2023-09-22 13:30:00 688799.SH   0.243225
                    688800.SH        NaN
                    688819.SH  -0.666667
                    688981.SH   0.065909
                    689009.SH   0.395312

[13669418 rows x 1 columns]

In [14]:
fac_df.index.get_label.between_time(start_time, end_time)

TypeError: Index must be DatetimeIndex

In [42]:
fac_df.isna().sum(), (~fac_df.isna()).sum()

(7240881, 17052437)

In [38]:
fac_df.loc[pd.to_datetime('2023-09-22 09:45:03')]

securityid
000001.SZ         NaN
000002.SZ    0.266224
000004.SZ    0.268894
000005.SZ   -0.109348
000006.SZ         NaN
               ...   
688799.SH   -0.279244
688800.SH   -0.377793
688819.SH    0.605374
688981.SH   -0.592932
689009.SH         NaN
Name: value, Length: 5059, dtype: float64

In [30]:
import alphalens

data=alphalens.utils.get_clean_factor_and_forward_returns(
    fac_df, prices, quantiles=5,periods=(20,60,100))

# alphalens.utils.get_clean_factor_and_forward_returns(fac_df, 
#     prices, 
#     quantiles=None,
#     bins=1,
#     periods=(1, 2, 3, 4, 5, 6, 10),
#     filter_zscore=None)


/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/alphalens/utils.py:158: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series(index=x.index)


Dropped 9.2% entries from factor data: 8.4% in forward returns computation and 0.9% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!


In [40]:
prices.loc[[pd.to_datetime('2023-09-22 11:30:00'), pd.to_datetime('2023-09-22 13:00:00'), pd.to_datetime('2023-09-22 13:01:00')]]

securityid,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,688787.SH,688788.SH,688789.SH,688793.SH,688798.SH,688799.SH,688800.SH,688819.SH,688981.SH,689009.SH
tradetime,,,,,,,,,,,,,,,,,,,,,
2023-09-22 11:30:00,11.14,13.29,16.260000,1.47,4.62,4.51,2.42,10.23,3.29,9.81,...,71.940002,37.369999,88.510002,35.720001,68.669998,40.009998,39.049999,34.419998,50.470001,34.799999
2023-09-22 13:00:00,11.14,13.30,16.260000,1.47,4.62,4.51,2.42,10.23,3.29,9.81,...,71.940002,37.369999,88.510002,35.720001,68.669998,40.009998,39.049999,34.419998,50.490002,34.799999
2023-09-22 13:01:00,11.13,13.28,16.290001,1.46,4.62,4.51,2.42,10.21,3.29,9.80,...,71.610001,37.389999,88.690002,35.730000,68.610001,39.930000,38.880001,34.400002,50.480000,34.849998


In [41]:
13.28/13.29-1

-0.0007524454477050302

In [33]:
data.loc[pd.to_datetime('2023-09-22 11:30:00')]

,1m,3m,5m,factor,factor_quantile
asset,,,,,
000001.SZ,-0.000898,-0.000898,-0.000898,0.001799,3.0
000002.SZ,-0.000752,-0.001505,-0.000752,0.003019,4.0
000004.SZ,0.001845,0.002460,0.003075,0.000615,2.0
000005.SZ,-0.006803,-0.013605,-0.013605,0.006849,5.0
000006.SZ,0.000000,0.000000,0.000000,0.000000,2.0
...,...,...,...,...,...
688799.SH,-0.001999,-0.003749,-0.002999,0.003763,4.0
688800.SH,-0.004353,-0.006914,-0.008963,0.006962,5.0
688819.SH,-0.000581,-0.001162,-0.002034,-0.000581,1.0


In [ ]:
prices.loc[pd.to_datetime('2023-09-22 11:29:57'):pd.to_datetime('2023-09-22 13:00:10')]

In [ ]:
import pandas as pd
# data.loc[pd.to_datetime('2023-09-22 11:29:57')]
data.loc[pd.to_datetime('2023-09-22 11:30:03'):pd.to_datetime('2023-09-22 11:30:30')]

In [ ]:
from alphalens import performance as perf
group_neutral = False
ic_data = perf.factor_information_coefficient(data, group_neutral)

import pandas as pd 
from scipy import stats
ic_summary_table = pd.DataFrame()
ic_summary_table["IC Mean"] = ic_data.mean()
ic_summary_table["IC Std."] = ic_data.std()
ic_summary_table["Risk-Adjusted IC"] = \
    ic_data.mean() / ic_data.std()
t_stat, p_value = stats.ttest_1samp(ic_data, 0, nan_policy='omit')
ic_summary_table["t-stat(IC)"] = t_stat
ic_summary_table["p-value(IC)"] = p_value
ic_summary_table["IC Skew"] = stats.skew(ic_data, nan_policy='omit')
ic_summary_table["IC Kurtosis"] = stats.kurtosis(ic_data, nan_policy='omit')
ic_summary_table['IC win rate'] = (ic_data > 0).sum() / ic_data.count()

In [ ]:
ic_summary_table

In [ ]:
alphalens.tears.create_returns_tear_sheet(data, long_short=True, group_neutral=False, by_group=False)

In [ ]:
alphalens.tears.create_information_tear_sheet(data, group_neutral=False, by_group=False)

In [ ]:
alphalens.tears.create_turnover_tear_sheet(data)

In [ ]:
alphalens.tears.create_summary_tear_sheet(data)

In [7]:
import re

def parse_function_call(s):
    def parse_args(s):
        args = []
        balance = 0
        current_arg = []
        for char in s:
            if char == ',' and balance == 0:
                args.append(''.join(current_arg).strip())
                current_arg = []
            else:
                if char == '(':
                    balance += 1
                elif char == ')':
                    balance -= 1
                current_arg.append(char)
        if current_arg:
            args.append(''.join(current_arg).strip())
        return args

    match = re.match(r'(\w+)\((.*)\)', s)
    if match:
        func_name = match.group(1)
        args_str = match.group(2)
        args = parse_args(args_str)
        args = [parse_function_call(arg) for arg in args]
        parsed_args = []
        parsed_kwargs = {}
        for arg in args:
            if isinstance(arg, dict):
                parsed_args.append(arg)
            elif isinstance(arg, str):
                arg = arg.strip()
                if "=" in arg:
                    parsed_kwargs[arg.split("=")[0].strip()] = arg.split("=")[1].strip()
                else:
                    parsed_args.append(arg)
        return {'function': func_name, 'args': parsed_args, 'kwargs': parsed_kwargs}
    else:
        return s

# 示例
input_str = "corr(add(x,mul(x,y)),z, window=1)"
parsed_result = parse_function_call(input_str)
print(parsed_result)

{'function': 'corr', 'args': [{'function': 'add', 'args': ['x', {'function': 'mul', 'args': ['x', 'y'], 'kwargs': {}}], 'kwargs': {}}, 'z'], 'kwargs': {'window': '1'}}


In [ ]:
def rank_pct(arr):
    sorter = np.argsort(arr)
    inv_sorter = np.argsort(sorter)
    ranks = np.arange(1, len(arr) + 1)
    pct_ranks = ranks[inv_sorter] / len(arr)
    return pct_ranks

# 示例
data = np.array([50, 30, 20, 40, 10])
result = rank_pct(data)
print(result)

In [23]:
import numpy as np
def rank_pct(arr):
    sorter = np.argsort(arr)
    inv_sorter = np.argsort(sorter)
    ranks = np.arange(1, len(arr) + 1)
    pct_ranks = ranks[inv_sorter] / len(arr)
    return pct_ranks

# 示例


In [25]:
sorter = np.argsort(data)
inv_sorter = np.argsort(sorter)
ranks = np.arange(1, len(arr) + 1)
pct_ranks = ranks[inv_sorter] / len(arr)

array([[4, 3, 1, 2, 0],
       [0, 2, 1, 3, 4]])

In [29]:
data = np.array([[50, 30, 20, 20, 10], [1,np.nan,2,4,5]])
sorter = np.argsort(data)
inv_sorter = np.argsort(sorter)
# result = rank_pct(data)
# print(result)
np.count_nonzero(~np.isnan(data), axis=1)

array([5, 4])

In [28]:
ranks = [np.arange(1, i+1) for i in np.count_nonzero(~np.isnan(data), axis=1)]
ranks

[array([1, 2, 3, 4, 5]), array([1, 2, 3, 4])]

In [9]:
arr = np.array([[1,2,3,4,5], [3,12,3,1,1]])
arr_df = pd.DataFrame(arr.T)
import time
start = time.time()
for i in range(100000):
    arr_df.rolling(window=3).mean(numeric_only=True, engine='numba')
print(time.time()-start)

start = time.time()
for i in range(100000):
    arr_df.rolling(window=3).mean()
print(time.time()-start)

5.013756990432739
8.345810651779175


In [21]:
np.count_nonzero(np.isnan(np.array([[np.nan, 1, 2], [3, np.nan, np.nan]])), axis=0)

array([1, 1, 1])

In [5]:
import numpy as np
import pandas as pd
a = np.array([[np.nan, 1, 2], [3, np.nan, np.nan], [2,3,4]])
a = pd.DataFrame(a)
a.ffill()

,0,1,2
0,NaN,1.0,2.0
1,3.0,1.0,2.0
2,2.0,3.0,4.0
